In [1]:
import time
import pandas as pd
import json
import urllib.request

from tqdm import tqdm
tqdm.pandas()

from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from math import ceil

In [21]:
def get_text_general(web_element) -> str:
    soup = BeautifulSoup(web_element.get_attribute('outerHTML'), "html.parser")
    paragraphs = soup.find_all('p')
    result = ''
    for paragraph in paragraphs:
        result += (paragraph.text.strip() + '\n')
    return result


In [31]:
def get_text_add(web_element) -> str:
    soup = BeautifulSoup(web_element.get_attribute('outerHTML'), "html.parser")
    paragraphs = soup.find_all('p')
    blocks = soup.find_all('ul')
    result = ''
    for i in range(len(paragraphs)):
        result += (paragraphs[i].text.strip() + '\n')
        try: 
            lines = blocks[i].find_all('li')
            for line in lines:
                result += (line.text.strip() + '\n')
        except:
            continue
    return result

In [34]:
def parse_text(link: str) -> str:
    try:
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')

        browser = webdriver.Chrome(options=options)
        browser.get(link)
        time.sleep(2)

        browser.find_element(By.XPATH, '/html/body/div[2]/div/section[2]/div/div[2]/div/div/div/div/section/div/nav/ul/li[1]/a/span').click()
        text = browser.find_element(By.XPATH, '/html/body/div[2]/div/section[2]/div/div[2]/div/div/div/div/section/div/div/section[1]/div/div/span')
        part_1 = get_text_general(text)
        time.sleep(1)

        browser.find_element(By.XPATH, '/html/body/div[2]/div/section[2]/div/div[2]/div/div/div/div/section/div/nav/ul/li[2]/a/span').click()
        text = browser.find_element(By.XPATH, '/html/body/div[2]/div/section[2]/div/div[2]/div/div/div/div/section/div/div/section[2]/div/div/span')
        part_2 = get_text_add(text)
        time.sleep(1)

        browser.quit()

        return part_1 + part_2
    except:
        return None

In [26]:
destination = '../dataoutput/nra_data.xlsx'
url = 'https://www.ra-national.ru/wp-load.php?security_key=100c906f36a0b90e&export_id=20&action=get_data'
urllib.request.urlretrieve(url, destination)

('nra_data.xlsx', <http.client.HTTPMessage at 0x1fbedf2fa90>)

In [27]:
df = pd.read_excel('../dataoutput/nra_data.xlsx')

In [35]:
df['text'] = df['Ссылка на пресс релиз'].progress_apply(parse_text)

100%|██████████| 312/312 [1:28:48<00:00, 17.08s/it]


In [40]:
df.sample(5)

,id,Название организации,ИНН,Название пресс-релиза,Дата опубликования пресс-релиза,Рейтинг,Статус рейтинга,Прогноз,Вид рейтинга,Сектор организации,Ссылка на пресс релиз,text
34,36487,АО «НПФ ГАЗФОНД пенсионные накопления»,7726445147,НРА подтвердило некредитный рейтинг надежности...,27.04.2023,AAA|ru.pf|,Подтверждён,Стабильный,Рейтинг надежности и качества услуг НПФ,Негосударственные пенсионные фонды,https://www.ra-national.ru/press_release/ao-np...,ПОДТВЕРЖДЕН НЕКРЕДИТНЫЙ РЕЙТИНГ\nОбщество с ог...
32,36491,АО «Дальневосточный банк»,2540016961,НРА подтвердило кредитный рейтинг АО «Дальнево...,28.04.2023,BBB|ru|,Подтверждён,Позитивный,Кредитный рейтинг кредитных организаций,Банки,https://www.ra-national.ru/press_release/ao-da...,ПОДТВЕРЖДЕН КРЕДИТНЫЙ РЕЙТИНГ\nОбщество с огра...
98,34876,АО «Страховая компания Гайде»,7809016423,НРА присвоило кредитный рейтинг АО «Страховая ...,07.10.2022,BB+|ru|,Присвоен,Стабильный,Кредитный рейтинг страховых компаний,Страховые компании,https://www.ra-national.ru/press_release/ao-gr...,Общество с ограниченной ответственностью «Наци...
57,36206,ЗПИФ недвижимости «ПНК-Рентал»,7703471817,НРА отозвало некредитный рейтинг надежности и ...,16.02.2023,—,Отозван,Без прогноза,Рейтинг надежности и инвестиционной привлекате...,ЗПИФ,https://www.ra-national.ru/press_release/zpif-...,Общество с ограниченной ответственностью «Наци...
29,36524,ВЕЛЕС ТРАСТ,7703603950,НРА отозвало некредитный рейтинг надежности и ...,04.05.2023,—,Отозван,Без прогноза,Рейтинг надежности и качества услуг УК,Управляющие компании,https://www.ra-national.ru/press_release/veles...,ОТОЗВАН НЕКРЕДИТНЫЙ РЕЙТИНГ\nОбщество с ограни...


In [39]:
df.to_csv('../data_output/nra_all.csv')